# Importando as Bibliotecas Necessárias

In [0]:
import time
import random
import math

# Caminhos

In [0]:
path_base_voos = '/content/drive/My Drive/Datasets/Otimização IA/voos.csv'

# Variáveis

In [0]:
#CWB: Aeroporto Internacional Afonso Pena - Curitiba
#GIG: Aeroporto Internacional Tom Jobim - Rio de Janeiro
#POA: Aeroporto Salgado Filho Porto Alegre - Porto Alegre
#FLN: Aeroporto Internacional de Florianópolis
#CNF: Aeroporto Internacional de Confins 
#GYN: Aeroporto de Goiania

pessoas = [('Amanda', 'CWB'),
           ('Pedro', 'GIG'),
           ('Marcos', 'POA'),
           ('Priscila', 'FLN'),
           ('Jessica', 'CNF'),
           ('Paulo', 'GYN')]

destino = 'GRU'

# Carregando a base de dados de voos e criando uma estrutura para consulta

In [0]:
# Criando um dicionario para armazenar todos os voos de uma determinada origem e um determinado destino
with open(path_base_voos, 'r') as arquivo_voos:
  linhas = arquivo_voos.readlines() 

voos = {}
for linha in linhas:()
  origem, destino, saida, chegada, preco = linha.strip().split(',')
  chave = (origem,destino)
  if chave in voos: 
    voos[chave].append((saida, chegada, float(preco)))
  else: 
    voos[chave] = [(saida, chegada, float(preco))]

In [0]:
# [1,4, 3,2, 7,3, 6,3, 2,4, 5,3]
def imprimir_agenda(agenda):
    id_voo = -1
    for i in range(len(agenda) // 2):
        nome = pessoas[i][0] 
        origem = pessoas[i][1]
        id_voo += 1
        ida = voos[(origem, destino)][agenda[id_voo]]
        id_voo += 1
        volta = voos[(destino, origem)][agenda[id_voo]]
        print(f'{nome:<15} - {origem} - {ida[0]:>5} - {ida[1]:>5} - R${ida[2]:>6.2f} - {volta[0]:>5} - {volta[1]:>5} - R${volta[2]:>6.2f}')


In [0]:
def get_minutos(hora):
    x = time.strptime(hora, '%H:%M')
    minutos = x[3] * 60 + x[4]
    return minutos

In [0]:
def funcao_custo(solucao):
    preco_total = 0
    ultima_chegada = 0
    primeira_partida = 1439
    
    id_voo = -1
    for i in range(len(solucao) // 2):
        origem = pessoas[i][1]
        id_voo += 1
        ida = voos[(origem, destino)][solucao[id_voo]]
        id_voo += 1
        volta = voos[(destino, origem)][solucao[id_voo]]
        
        preco_total += ida[2]
        preco_total += volta[2]
        
        if ultima_chegada < get_minutos(ida[1]):
            ultima_chegada = get_minutos(ida[1])
        if primeira_partida > get_minutos(volta[0]):
            primeira_partida = get_minutos(volta[0])
            
    
    id_voo = -1
    total_espera = 0    
    for i in range(len(solucao) // 2):
        origem = pessoas[i][1]
        id_voo += 1
        ida = voos[(origem, destino)][solucao[id_voo]]
        id_voo += 1
        volta = voos[(destino, origem)][solucao[id_voo]]
        
        total_espera += ultima_chegada - get_minutos(ida[1])
        total_espera += get_minutos(volta[0]) - primeira_partida
        
    if ultima_chegada > primeira_partida:
        preco_total += 500
        
    return preco_total + total_espera * 10


In [351]:
solucao = [1,2, 3,2, 5,3, 4,3, 2,4, 5,3]
funcao_custo(solucao)

12933.0

# Busca Aleatória

In [0]:
def pesquisa_randomica(dominio, funcao_custo):
    melhor_custo = 999999999
    melhor_solucao = None
    for i in range(0, 10000):
        solucao = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]
        custo = funcao_custo(solucao)
        if custo < melhor_custo:
            melhor_custo = custo
            melhor_solucao = solucao
    return melhor_solucao


In [191]:
dominio = [(0,9)] * (len(pessoas) * 2)
solucao_randomica = pesquisa_randomica(dominio, funcao_custo)
custo_randomica = funcao_custo(solucao_randomica)
imprimir_agenda(solucao_randomica)
print('Custo da solução:', custo_randomica)

Amanda          - CWB - 15:27 - 17:18 - R$151.00 - 12:08 - 14:05 - R$142.00
Pedro           - GIG - 12:19 - 15:25 - R$342.00 - 12:20 - 16:34 - R$500.00
Marcos          - POA - 17:08 - 19:08 - R$262.00 - 12:01 - 13:41 - R$267.00
Priscila        - FLN - 17:07 - 20:04 - R$291.00 - 15:23 - 18:49 - R$150.00
Jessica         - CNF - 16:43 - 19:00 - R$246.00 - 12:08 - 14:47 - R$231.00
Paulo           - GYN - 18:12 - 20:17 - R$242.00 - 16:35 - 18:56 - R$144.00
Custo da solução: 14858.0


# Hill Climbing

In [0]:
def subida_encosta(dominio, funcao_custo):
    
    solucao = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]
    atual = solucao
    melhor = funcao_custo(solucao)

    cont = 0
    while True:
      cont = cont + 1
      vizinhos = []
      
      for i in range(len(dominio)):
          if solucao[i] >= dominio[i][0]:
              if solucao[i] < dominio[i][1]:
                  vizinhos.append(solucao[0:i] + [solucao[i] + 1] + solucao[i + 1:])
          if solucao[i] <= dominio[i][1]:
              if solucao[i] > dominio[i][0]:
                  vizinhos.append(solucao[0:i] + [solucao[i] - 1] + solucao[i + 1:])
      
      for i in range(len(vizinhos)):
          custo = funcao_custo(vizinhos[i])
          if custo < melhor:
              melhor = custo
              solucao = vizinhos[i]
              
      if melhor == atual:
          break
      else:
          atual = melhor

    #print('Num iterações: ', cont)
    return solucao

In [234]:
solucao_subida_encosta = subida_encosta(dominio, funcao_custo)
custo_subida_encosta = funcao_custo(solucao_subida_encosta)
imprimir_agenda(solucao_subida_encosta)
print('Custo da solução:', custo_subida_encosta)

Num iterações:  27
Amanda          - CWB - 12:34 - 15:02 - R$109.00 -  9:58 - 11:18 - R$130.00
Pedro           - GIG - 10:30 - 14:57 - R$290.00 -  9:49 - 13:51 - R$229.00
Marcos          - POA - 12:08 - 14:59 - R$149.00 -  9:58 - 12:56 - R$249.00
Priscila        - FLN - 11:28 - 14:40 - R$248.00 -  9:25 - 12:46 - R$295.00
Jessica         - CNF - 12:44 - 14:17 - R$134.00 - 10:33 - 13:11 - R$132.00
Paulo           - GYN - 12:18 - 14:56 - R$172.00 -  9:31 - 11:43 - R$210.00
Custo da solução: 5297.0


# Simullated Annealing

In [0]:
def tempera_simulada(dominio, funcao_custo, temperatura = 100000.0, resfriamento = 0.95):
    solucao = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]

    incremento = [-1,1]
    while temperatura > 0.1:
        i = random.randint(0, len(dominio) - 1) 
        direcao = random.randint(0, 1)
        
        solucao_temp = solucao[:]
        solucao_temp[i] += incremento[direcao]
        if solucao_temp[i] < dominio[i][0]:
            solucao_temp[i] = dominio[i][0]
        elif solucao_temp[i] > dominio[i][1]:
            solucao_temp[i] = dominio[i][1]
            
        custo_solucao = funcao_custo(solucao)
        custo_solucao_temp = funcao_custo(solucao_temp)
        probabilidade = pow(math.e, (- abs(custo_solucao_temp - custo_solucao)) / temperatura)
        
        if (custo_solucao_temp < custo_solucao or random.random() < probabilidade):
            solucao = solucao_temp
        
        temperatura = temperatura * resfriamento
    return solucao

In [294]:
solucao_tempera_simulada = tempera_simulada(dominio, funcao_custo)
custo_tempera_simulada = funcao_custo(solucao_tempera_simulada)
imprimir_agenda(solucao_tempera_simulada)
print('Custo da solução:', custo_tempera_simulada)

Amanda          - CWB - 17:11 - 18:30 - R$108.00 - 10:33 - 12:03 - R$ 74.00
Pedro           - GIG - 15:44 - 18:55 - R$382.00 -  7:57 - 11:15 - R$347.00
Marcos          - POA - 17:08 - 19:08 - R$262.00 -  8:19 - 11:16 - R$122.00
Priscila        - FLN - 15:34 - 18:11 - R$326.00 -  8:23 - 11:07 - R$143.00
Jessica         - CNF - 16:43 - 19:00 - R$246.00 -  7:50 - 10:08 - R$164.00
Paulo           - GYN - 16:51 - 19:09 - R$147.00 -  8:04 - 10:59 - R$136.00
Custo da solução: 6627.0


# Algoritmo Genético

In [0]:
def cruzamento(solucao1, solucao2):
    i = random.randint(1, len(solucao1) - 2)
    return solucao1[:i] + solucao2[i:], solucao2[:i] + solucao1[i:]

In [0]:
def mutacao(dominio, solucao, taxa_mutacao):
  incremento = [-1,1]
  mutante = solucao[:]
  
  for i in range(len(solucao)):
    if random.random() < taxa_mutacao: 
      direcao = random.randint(0, 1)
      novo_valor = mutante[i] + incremento[direcao]

      if (novo_valor >= dominio[i][0] and novo_valor <= dominio[i][1]):
        mutante[i] = novo_valor

  return mutante

In [0]:
def selecao_roleta(populacao):
  soma_custos = 0
  custos = []
  for individuo in populacao: 
    fitness_individuo = 1 / funcao_custo(individuo)
    custos.append((fitness_individuo, individuo))
    soma_custos += fitness_individuo 

  acm = 0
  roleta = []
  for elem in custos: 
    atual = elem[0] / soma_custos
    acm += atual 
    roleta.append(acm)

  selecionados = []
  while len(selecionados) < len(populacao):
    prob = random.random();
    for i in range(0,len(populacao)):
      if prob < roleta[i]:
        break
    selecionados.append(custos[i][1])
  
  return selecionados
  
  

In [0]:
def selecao_elitista(populacao):
  elitismo = 0.2
  numero_elitismo = int(elitismo * len(populacao))
  custos = [(funcao_custo(individuo), individuo) for individuo in populacao]
  custos.sort()
  individuos_ordenados = [individuo for (custo, individuo) in custos]
  
  return individuos_ordenados[0:numero_elitismo]

In [0]:
def genetico(dominio, funcao_custo, tamanho_populacao = 100, 
             taxa_mutacao = 0.2, numero_geracoes = 500):
    
    populacao = []
    for i in range(tamanho_populacao):
        solucao = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]
        populacao.append(solucao)
    
    melhor_individuo = None
    melhor_custo = 900000000000

    for i in range(numero_geracoes):
        #print('Geracao: ', i)
        #selecionados = selecao_elitista(populacao)        
        selecionados = selecao_roleta(populacao)        
        
        nova_populacao = []
        while len(nova_populacao) < tamanho_populacao:
            c1 = random.randrange(0, len(selecionados))
            c2 = random.randrange(0, len(selecionados))
            nc1,nc2 = cruzamento(selecionados[c1], selecionados[c2])
            nc1 = mutacao(dominio,nc1,taxa_mutacao)
            nc2 = mutacao(dominio,nc2,taxa_mutacao)
            custo_nc1 = funcao_custo(nc1)
            custo_nc2 = funcao_custo(nc2)

            if custo_nc1 < melhor_custo:
              melhor_custo = custo_nc1 
              melhor_individuo = nc1
            if custo_nc2 < melhor_custo:
              melhor_custo = custo_nc2 
              melhor_individuo = nc2  

            nova_populacao.append(nc1)
            nova_populacao.append(nc2)

        populacao = nova_populacao[:]
        #print('Melhor custo:', melhor_custo)

    return melhor_individuo

In [461]:
solucao_genetico = genetico(dominio, funcao_custo)
custo_genetico = funcao_custo(solucao_genetico)
imprimir_agenda(solucao_genetico)
print('Custo da solução:', custo_genetico)


Geracao:  0
Melhor custo: 22045.0
Geracao:  1
Melhor custo: 14608.0
Geracao:  2
Melhor custo: 13094.0
Geracao:  3
Melhor custo: 13094.0
Geracao:  4
Melhor custo: 12505.0
Geracao:  5
Melhor custo: 12230.0
Geracao:  6
Melhor custo: 11754.0
Geracao:  7
Melhor custo: 10393.0
Geracao:  8
Melhor custo: 8976.0
Geracao:  9
Melhor custo: 8695.0
Geracao:  10
Melhor custo: 6958.0
Geracao:  11
Melhor custo: 6958.0
Geracao:  12
Melhor custo: 6958.0
Geracao:  13
Melhor custo: 6711.0
Geracao:  14
Melhor custo: 6711.0
Geracao:  15
Melhor custo: 6711.0
Geracao:  16
Melhor custo: 6711.0
Geracao:  17
Melhor custo: 6711.0
Geracao:  18
Melhor custo: 6711.0
Geracao:  19
Melhor custo: 6711.0
Geracao:  20
Melhor custo: 6711.0
Geracao:  21
Melhor custo: 6711.0
Geracao:  22
Melhor custo: 6711.0
Geracao:  23
Melhor custo: 6711.0
Geracao:  24
Melhor custo: 6711.0
Geracao:  25
Melhor custo: 6711.0
Geracao:  26
Melhor custo: 6711.0
Geracao:  27
Melhor custo: 6711.0
Geracao:  28
Melhor custo: 6711.0
Geracao:  29
Mel

# Área de Testes

In [0]:
num_repeticoes = 20;
resultados_hill = [funcao_custo(subida_encosta(dominio, funcao_custo)) for i in range(0,num_repeticoes)]
resultados_annealing = [funcao_custo(tempera_simulada(dominio, funcao_custo)) for i in range(0,num_repeticoes)]
resultados_genetico = [funcao_custo(genetico(dominio, funcao_custo)) for i in range(0,num_repeticoes)]

In [0]:
resultados_random = [funcao_custo(pesquisa_randomica(dominio, funcao_custo)) for i in range(0,num_repeticoes)]

In [0]:
import pandas as pd

In [0]:
resultados = pd.DataFrame({'Random': resultados_random,
                           'Hill Climbing':resultados_hill,
                           'Sim. Annealing': resultados_annealing, 
                           'Alg. Genético': resultados_genetico})

In [479]:
resultados

,Random,Hill Climbing,Sim. Annealing,Alg. Genético
0,11778.0,7347.0,6900.0,7366.0
1,13529.0,4366.0,6285.0,5402.0
2,15344.0,5135.0,7320.0,6711.0
3,13337.0,5837.0,7926.0,6555.0
4,12628.0,7291.0,13415.0,6711.0
5,13265.0,6018.0,7303.0,6775.0
6,14836.0,7479.0,6598.0,6742.0
7,11214.0,5695.0,11868.0,6346.0
8,14091.0,4366.0,8230.0,5635.0
9,11920.0,6134.0,13472.0,6134.0


In [480]:
resultados.describe()

,Random,Hill Climbing,Sim. Annealing,Alg. Genético
count,20.000000,20.00000,20.000000,20.000000
mean,13858.750000,6202.10000,8185.100000,6418.750000
std,1449.037317,1192.44397,2523.995702,483.521934
min,11214.000000,4366.00000,5135.000000,5402.000000
25%,12874.750000,5502.50000,6554.000000,6134.000000
50%,13599.500000,6066.00000,7397.500000,6555.000000
75%,15205.250000,7305.00000,8247.500000,6711.000000
max,16089.000000,7951.00000,13472.000000,7366.000000
